In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
import matplotlib.pyplot as plt
import holidays
import seaborn as sns
from sklearn import linear_model
sns.set()
%matplotlib inline 

C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Feature engineering

Since we want to apply classification model, we will create new attributes from the dataset so that the models can learn. 
<br>

Based on the exploratory analysis, the following features would be relevant:
- month
- dayofweek
- holiday vs normal day
- product category
- shop id


However, the test set for the submission is asking us to submit at total month level, so the dayofweek and holiday features would be applicable. We left with month, product category and shop id

In [2]:
raw_df = pd.read_csv('data/train_df.csv', parse_dates=['date'])
# outlier removal steps as identified in exploratory analysis
id_out = [11, 14, 15, 12]
raw_df.loc[(raw_df.date == '2013-11-29') & (raw_df.item_category_id.isin(id_out)), 'sales'] = np.nan
raw_df.loc[(raw_df.date_block_num == 10) & (raw_df.item_category_id.isin(id_out))].interpolate(method='linear');
raw_df.head()

,date,date_block_num,shop_id,shop_name,item_id,item_name,item_category_id,item_category_name,item_price,item_cnt_day,sales
0,2013-01-02,0,59,"Ярославль ТЦ ""Альтаир""",22154,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,999.0,1.0,999.0
1,2013-01-02,0,25,"Москва ТРК ""Атриум""",22154,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,999.0,1.0,999.0
2,2013-01-03,0,25,"Москва ТРК ""Атриум""",22154,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,999.0,1.0,999.0
3,2013-01-20,0,25,"Москва ТРК ""Атриум""",22154,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,999.0,1.0,999.0
4,2013-01-23,0,25,"Москва ТРК ""Атриум""",22154,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,999.0,1.0,999.0


In [3]:
raw_df['month'] = raw_df['date'].dt.month

In [4]:
# daily level info would not match the test data set, hence groupby month
month_df = raw_df.groupby(['date_block_num', 'month', 'shop_id', 'item_category_id', 'item_id'])['item_cnt_day'].sum().reset_index()

In [5]:
month_df.head()

,date_block_num,month,shop_id,item_category_id,item_id,item_cnt_day
0,0,1,0,2,5572,10.0
1,0,1,0,2,5573,1.0
2,0,1,0,2,5575,4.0
3,0,1,0,2,5576,5.0
4,0,1,0,2,5609,1.0


In [6]:
df = pd.get_dummies(month_df, columns=['shop_id', 'item_category_id', 'month'])

In [7]:
train = df[month_df.date_block_num != 33]
test = df[month_df.date_block_num == 33]

In [8]:
train_f = train.drop(['item_cnt_day'], axis=1)
train_t = train['item_cnt_day']

In [9]:
test_f = test.drop(['item_cnt_day'], axis=1)
test_t = test['item_cnt_day']

In [10]:
DM_train = xgb.DMatrix(data=train_f, label=train_t)
DM_test =  xgb.DMatrix(data=test_f, label=test_t)

In [11]:
params = {"booster":"gblinear", "objective":"reg:squarederror"}

In [12]:
xg_reg = xgb.train(params = params, dtrain=DM_train, num_boost_round=5)

In [13]:
preds = xg_reg.predict(DM_test)

rmse = np.sqrt(mean_squared_error(test_t,preds))
print("RMSE: %f" % (rmse))

RMSE: 13.768398


In [14]:
# model = XGBRegressor(
#     max_depth=8,
#     n_estimators=100,
#     min_child_weight=30, 
#     colsample_bytree=0.8, 
#     subsample=0.8, 
#     eta=0.3,    
#     seed=42)

In [15]:
# model.fit(train_f, train_t)

In [16]:
# predict = model.predict(test_f)
# rmse_1 = np.sqrt(mean_squared_error(test_t, predict))
# rmse_1

In [17]:
rfc = RandomForestRegressor()

In [18]:
rfc.fit(train_f, train_t)

KeyboardInterrupt: 

In [ ]:
pred_rfc = rfc.predict(test_f)

In [19]:
regr = linear_model.LinearRegression()
regr.fit(train_f, train_t)
pred_reg = regr.predict(test_f)
rmse_reg = np.sqrt(mean_squared_error(test_t, pred_reg))
print("RMSE: %f" % (rmse_reg))

RMSE: 13.770137
